# Pre-processamento das imagens
<p>Essa etapa arrumamos as imagens removendo das bordas, aumentando o contraste e reduzindo o tamanho para 224x224px</p>

In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
import torch

import matplotlib.pyplot as plt
import cv2

from tqdm.notebook import tqdm

import torchvision.transforms as transforms

In [11]:
#DataSet Original
PATH_DS_TRAINING_IMG = Path("./OIA-ODIR/Training Set/Images")
PATH_DS_VALIDATION_IMG = Path("./OIA-ODIR/Off-site Test Set/Images")
PATH_DS_TEST_IMG = Path("./OIA-ODIR/On-site Test Set/Images")

#Pasta para salvar o Dataset para treinamento e validação
PATH_DS_PROCESSED_TRAINING = Path("./IMG224_ALL/PROCESSED-TRAIN/")
PATH_DS_PROCESSED_VALIDATION = Path("./IMG224_ALL/PROCESSED-VALID/")
PATH_DS_PROCESSED_TEST = Path("./IMG224_ALL/PROCESSED-TEST/")

ds_training = pd.read_excel("training_final.xlsx")
ds_validation = pd.read_excel("validation_final.xlsx")
ds_test = pd.read_excel("test_final.xlsx")

In [12]:
def GetLabelNumber(line):
    """
        0-N - Normal
        1-D - Diabetes
        2-G - Glaucoma
        3-C - Catarata
        4-A - Degeneração macular
        5-H - Hipertensao
        6-M - Miopia patológica 
    """
    if(line.N==1):
        return "0"
    if(line.D==1):
        return "1"
    if(line.G==1):
        return "2"
    if(line.C==1):
        return "3"
    if(line.A==1):
        return "4"
    if(line.H==1):
        return "5"
    if(line.M==1):
        return "6"
    
    return "ERR"
    

In [13]:
def GetLabelBiclassNumber(line):
    """
        0- Normal
        1- No Normal
    """
    if(line.N==1):
        return "0"
    
    return "1"
    

In [14]:
def EncontrarMargemEsquerda(img):
    altura, largura = img.shape[:2]
    centro = altura // 2
    for pixel in range(largura):
        if img[centro][pixel][0] > 10 or img[centro][pixel][1] > 10 or img[centro][pixel][2] > 10:
            return pixel
            break
    
    return 0
    
def EncontrarMargemDireita(img):
    altura, largura = img.shape[:2]
    largura = largura - 1
    centro = altura // 2
    for pixel in range(largura, -1, -1):
        if img[centro][pixel][0] > 10 or img[centro][pixel][1] > 10 or img[centro][pixel][2] > 10:
            return pixel
            break
    
    return 0

def EncontrarMargemSuperior(img):
    altura, largura = img.shape[:2]
    centro = largura // 2
    for pixel in range(altura):
        if img[pixel][centro][0] > 10 or img[pixel][centro][1] > 10 or img[pixel][centro][2] > 10:
            return pixel
            break
    
    return 0

def EncontrarMargemInferior(img):
    altura, largura = img.shape[:2]
    altura = altura - 1
    centro = largura // 2
    for pixel in range(altura, -1, -1):
        if img[pixel][centro][0] > 10 or img[pixel][centro][1] > 10 or img[pixel][centro][2] > 10:
            return pixel
            break
    
    return 0
    
def RemoverMargens(img):
    margemEsq = EncontrarMargemEsquerda(img)
    margemDir = EncontrarMargemDireita(img)
    margemSup = EncontrarMargemSuperior(img)
    margemInf = EncontrarMargemInferior(img)
    return img[margemSup: margemInf, margemEsq:margemDir]
    

In [15]:
IMG_HEIGHT = 224
IMG_WIDTH = 224

def AjusteContraste(img):
    img2 = cv2.cvtColor(img, cv2.COLOR_BGR2Lab)
    
    clahe = cv2.createCLAHE(clipLimit=2,tileGridSize=(5,5))
    
    img2[:,:,0] = clahe.apply(img2[:,:,0])
    img2[:,0,:] = clahe.apply(img2[:,0,:])
    img2[0,:,:] = clahe.apply(img2[0,:,:])
    
    img2 = cv2.cvtColor(img2, cv2.COLOR_Lab2RGB)
    return img2

def PreprocessarImagem(img):
    
    img = RemoverMargens(img)
    img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT), interpolation=cv2.INTER_LANCZOS4)
    
    img = AjusteContraste(img)
    
    return img
    
def PreprocessarImagensDataset(dataset, path_ds, jpg, bi_class, path_ds_processed, train):
    i_arr = range(dataset.shape[0])

    for c, i in enumerate(tqdm(i_arr)):
        
        item = dataset.iloc[i]
        
        file_name = item.file_name
        side_eye = item.side
        
        img_path = path_ds / file_name
        img = cv2.imread(str(img_path))
        
        img = PreprocessarImagem(img)
        
        if jpg:
            if bi_class:
                save_path = path_ds_processed/GetLabelBiclassNumber(item)
            else:
                label = GetLabelNumber(item)
                save_path = path_ds_processed/label
            
            save_path.mkdir(parents=True, exist_ok=True)
            file_fullname = str(save_path/file_name)
            
            cv2.imwrite(file_fullname, img)
            

            if label == "1":
                img9 = train_transforms(img)
                img9 = img9.permute(1, 2, 0)
                img9 = img9 * 255
                img9 = img9.int()
                img9 = img9.numpy()
                file_name = "9_" + file_name
                file_fullname = str(save_path / file_name)
                
                cv2.imwrite(file_fullname, img9)


            if label == "2" or label == "3" or label == "4" or label == "5" or label == "6":
                img9 = train_transforms(img)
                img9 = img9.permute(1, 2, 0)
                img9 = img9 * 255
                img9 = img9.int()
                img9 = img9.numpy()
                file_name1 = "9_" + file_name
                file_fullname = str(save_path / file_name1)
                cv2.imwrite(file_fullname, img9)

                img9 = train_transforms(img)
                img9 = img9.permute(1, 2, 0)
                img9 = img9 * 255
                img9 = img9.int()
                img9 = img9.numpy()
                file_name1 = "8_" + file_name
                file_fullname = str(save_path / file_name1)
                cv2.imwrite(file_fullname, img9)
                
                img9 = train_transforms(img)
                img9 = img9.permute(1, 2, 0)
                img9 = img9 * 255
                img9 = img9.int()
                img9 = img9.numpy()
                file_name1 = "7_" + file_name
                file_fullname = str(save_path / file_name1)
                cv2.imwrite(file_fullname, img9)

        else:
            img_arr = img.astype(np.float16)

            if bi_class:
                save_path = path_ds_processed/GetLabelBiclassNumber(item)
            else:
                label = GetLabelNumber(item)
                save_path = path_ds_processed/label

            save_path.mkdir(parents=True, exist_ok=True)
            np.save(save_path/file_name, img_arr)
            
            if label == "1":
                img9 = train_transforms(img)
                img9 = img9.permute(1, 2, 0)
                img9 = img9 * 255
                img9 = img9.int()
                img9 = img9.numpy()
                
                img_arr = img9.astype(np.float16)
                file_name = "9_" + file_name
                file_fullname = str(save_path / file_name)
                np.save(save_path/file_name, img_arr)


            if label == "2" or label == "3" or label == "4" or label == "5" or label == "6":
                img9 = train_transforms(img)
                img9 = img9.permute(1, 2, 0)
                img9 = img9 * 255
                img9 = img9.int()
                img9 = img9.numpy()
                
                img_arr = img9.astype(np.float16)
                file_name = "9_" + file_name
                file_fullname = str(save_path / file_name)
                np.save(save_path/file_name, img_arr)
                
                img9 = train_transforms(img)
                img9 = img9.permute(1, 2, 0)
                img9 = img9 * 255
                img9 = img9.int()
                img9 = img9.numpy()
                
                img_arr = img9.astype(np.float16)
                file_name = "8_" + file_name
                file_fullname = str(save_path / file_name)
                np.save(save_path/file_name, img_arr)
                
                img9 = train_transforms(img)
                img9 = img9.permute(1, 2, 0)
                img9 = img9 * 255
                img9 = img9.int()
                img9 = img9.numpy()
                
                img_arr = img9.astype(np.float16)
                file_name = "7_" + file_name
                file_fullname = str(save_path / file_name)
                np.save(save_path/file_name, img_arr)
            
            
        
        

In [16]:
IMG_HEIGHT = 224
IMG_WIDTH = 224

def AjusteContraste(img):
    img2 = cv2.cvtColor(img, cv2.COLOR_BGR2Lab)
    
    clahe = cv2.createCLAHE(clipLimit=2,tileGridSize=(5,5))
    
    img2[:,:,0] = clahe.apply(img2[:,:,0])
    img2[:,0,:] = clahe.apply(img2[:,0,:])
    img2[0,:,:] = clahe.apply(img2[0,:,:])
    
    img2 = cv2.cvtColor(img2, cv2.COLOR_Lab2RGB)
    return img2

def PreprocessarImagem(img):
    
    img = RemoverMargens(img)
    img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT), interpolation=cv2.INTER_LANCZOS4)
    
    img = AjusteContraste(img)
    
    return img
    
def PreprocessarImagensDataset(dataset, path_ds, jpg, bi_class, path_ds_processed, train):   
    i_arr = range(dataset.shape[0])

    for c, i in enumerate(tqdm(i_arr)):
        
        item = dataset.iloc[i]
        
        file_name = item.file_name
        side_eye = item.side
        
        img_path = path_ds / file_name
        img = cv2.imread(str(img_path))
        
        img = PreprocessarImagem(img)
        
        if jpg:
            if bi_class:
                save_path = path_ds_processed/GetLabelBiclassNumber(item)
            else:
                label = GetLabelNumber(item)
                save_path = path_ds_processed/label
            
            save_path.mkdir(parents=True, exist_ok=True)
            file_fullname = str(save_path/file_name)
            
            cv2.imwrite(file_fullname, img)
            
        else:
            img_arr = img.astype(np.float16)

            if bi_class:
                save_path = path_ds_processed/GetLabelBiclassNumber(item)
            else:
                label = GetLabelNumber(item)
                save_path = path_ds_processed/label
            
            save_path.mkdir(parents=True, exist_ok=True)
            np.save(save_path/file_name, img_arr)
 
        

### Depois de executar o préprocessamento das imagens serão criadas as pastas 0, 1, 2, 3, 4, 5 e 6 dentro de "IMG224_ALL" referentes as classificações  Normal, Diabetes, Glaucoma, Catarata, Degeneração macular, Hipertensao e Miopia patológica.

#### Se voce não pretende treinar a classificação de umas dessas classes é necessaio excluir a pasta correspondente.

In [17]:
PreprocessarImagensDataset(ds_training, PATH_DS_TRAINING_IMG, False, False, PATH_DS_PROCESSED_TRAINING, True)

  0%|          | 0/5098 [00:00<?, ?it/s]

In [18]:
PreprocessarImagensDataset(ds_validation,PATH_DS_VALIDATION_IMG, False, False, PATH_DS_PROCESSED_VALIDATION, False)

  0%|          | 0/717 [00:00<?, ?it/s]

In [19]:
PreprocessarImagensDataset(ds_test,PATH_DS_TEST_IMG,False, False, PATH_DS_PROCESSED_TEST, False)

  0%|          | 0/1386 [00:00<?, ?it/s]

### Não esqueça de excluir as pastas correspondentes as classificações que não serão utilizadas.